In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,imbituba,-28.24,-48.67,82.40,61,20,10.29,BR,1605364003
1,1,bugiri,0.57,33.74,75.81,78,73,4.65,UG,1605364003
2,2,bredasdorp,-34.53,20.04,64.40,59,100,19.46,ZA,1605364003
3,3,muriwai beach,-36.82,174.45,54.00,93,0,4.18,NZ,1605364004
4,4,ilulissat,69.22,-51.10,12.20,66,75,24.16,GL,1605364004


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
202,202,shadegan,30.65,48.66,77.00,38,0,9.17,IR,1605364018
219,219,buraydah,26.33,43.98,75.20,47,0,8.05,SA,1605364019
328,328,jaisalmer,26.91,70.92,77.50,23,0,7.58,IN,1605363947
368,368,poya,-21.35,165.15,74.55,84,0,3.53,NC,1605364029
441,441,waddan,29.16,16.14,73.47,35,0,7.74,LY,1605364033
537,537,palma,39.57,2.65,73.40,64,0,6.93,ES,1605363744


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
202,shadegan,IR,30.65,48.66,
219,buraydah,SA,26.33,43.98,
328,jaisalmer,IN,26.91,70.92,
368,poya,NC,-21.35,165.15,
441,waddan,LY,29.16,16.14,
537,palma,ES,39.57,2.65,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
202,shadegan,IR,30.65,48.66,ديوانية شيخ عبود الذياب آل سالم
219,buraydah,SA,26.33,43.98,Mövenpick Hotel Qassim
328,jaisalmer,IN,26.91,70.92,Hotel Fifu
368,poya,NC,-21.35,165.15,
441,waddan,LY,29.16,16.14,محل الشريف محمد العايب
537,palma,ES,39.57,2.65,Hotel Saratoga


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))